In [42]:
from textblob import TextBlob

In [43]:
import pandas as pd
corpus = pd.read_pickle('SST2.pkl')
sentences, labels = list(corpus.sentence), list(corpus.label)

In [44]:
from keras.layers.core import Activation, Dense, Dropout, SpatialDropout1D
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.layers import Bidirectional
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
import collections
import matplotlib.pyplot as plt
import nltk
import numpy as np



In [50]:
max_len = 0
word_freqs = collections.Counter()
word_polarity=dict()
num_recs = len(sentences)
label_list=[]
for index,sentence in enumerate(sentences):
    label_list.append(labels[index])
    words=nltk.word_tokenize(sentence.lower())
    pos_tags=nltk.pos_tag(words)
    word2tag={ item[0]:item[1] for item in pos_tags}
#     print(word2tag)
    if(len(words)>max_len):
        max_len=len(words)
    for word in words:
        if word2tag[word] in ['JJ','JJS','JJS','RB','RBR','RBS']:
#             print(word)
            word_freqs[word]+=1
            word_polarity[word]=TextBlob(word).sentiment.polarity
nos_classes=len(list(set(labels)))
label_set_list=list(set(labels))
MAX_FEATURES=7000
MAX_SENTENCE_LENGTH=30

In [51]:
vocab_size = min(MAX_FEATURES, len(word_freqs)) + 2
word2polarity = {x[0]: word_polarity[x[0]] for i, x in
enumerate(word_freqs.most_common(MAX_FEATURES))}
word2polarity["PAD"]=0
word2polarity["UNK"]=0
# index2word = {v:k for k, v in word2index.items()}

In [52]:
X = np.empty((num_recs, ), dtype=list)
nos_classes=2
y = np.zeros((num_recs, nos_classes))
i = 0
# f_train = open(os.path.join(current_dir, file_name), 'rb')
for index,sentence in enumerate(sentences):
    l=[0]*nos_classes
    seqs=[]
    words=nltk.word_tokenize(sentence.lower())
    for word in words:
        if word2tag[word] in ['JJ','JJS','JJS','RB','RBR','RBS']:
            if word in word2polarity.keys():
                seqs.append(word2polarity[word])
            else:
                seqs.append(word2polarity['UNK'])
    X[i]=seqs
    l[label_set_list.index(labels[index])]=1
    y[i]=l
    i+=1
MAX_SENTENCE_LENGTH=80
X=sequence.pad_sequences(X,maxlen=MAX_SENTENCE_LENGTH)

KeyError: 'new'

In [33]:
Xtrain,Xtest,ytrain,ytest=train_test_split(X,y,test_size=0.2)
print(Xtrain.shape)

(63723, 80)


In [39]:
from keras.layers import Reshape
model = Sequential()
# model.add(Embedding(vocab_size, MAX_SENTENCE_LENGTH, input_length=MAX_SENTENCE_LENGTH))
model.add(Dense(MAX_SENTENCE_LENGTH, input_dim=MAX_SENTENCE_LENGTH, init='glorot_uniform', activation='relu')) 
model.add(Reshape((MAX_SENTENCE_LENGTH, 1)))
model.add(Conv1D(filters=100, kernel_size=10, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=5))
model.add(Conv1D(filters=100, kernel_size=10, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=5))
model.add(Conv1D(filters=32, kernel_size=10, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

/Users/gowthamkannan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(80, input_dim=80, activation="relu", kernel_initializer="glorot_uniform")`
  after removing the cwd from sys.path.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 80)                6480      
_________________________________________________________________
reshape_1 (Reshape)          (None, 80, 1)             0         
_________________________________________________________________
conv1d_21 (Conv1D)           (None, 80, 100)           1100      
_________________________________________________________________
max_pooling1d_13 (MaxPooling (None, 16, 100)           0         
_________________________________________________________________
conv1d_22 (Conv1D)           (None, 16, 100)           100100    
_________________________________________________________________
max_pooling1d_14 (MaxPooling (None, 3, 100)            0         
_________________________________________________________________
conv1d_23 (Conv1D)           (None, 3, 32)             32032     
__________

In [ ]:
BATCH_SIZE=75
NUM_EPOCHS=5
history=model.fit(Xtrain,ytrain,batch_size=BATCH_SIZE,epochs=NUM_EPOCHS,validation_data=(Xtest,ytest))

Train on 63723 samples, validate on 15931 samples
Epoch 1/5
63723/63723 [==============================] - 66s 1ms/step - loss: 0.6800 - acc: 0.5501 - val_loss: 0.6766 - val_acc: 0.5591
Epoch 2/5
63723/63723 [==============================] - 64s 1ms/step - loss: 0.6745 - acc: 0.5591 - val_loss: 0.6752 - val_acc: 0.5590
Epoch 3/5
63723/63723 [==============================] - 63s 990us/step - loss: 0.6740 - acc: 0.5590 - val_loss: 0.6753 - val_acc: 0.5592
Epoch 4/5
63723/63723 [==============================] - 65s 1ms/step - loss: 0.6739 - acc: 0.5594 - val_loss: 0.6752 - val_acc: 0.5593
Epoch 5/5
63675/63723 [============================>.] - ETA: 0s - loss: 0.6735 - acc: 0.5594

In [ ]:
plt.subplot(211)
plt.title("Accuracy")
plt.plot(history.history["acc"], color="g", label="Train")
plt.plot(history.history["val_acc"], color="b", label="Validation")
plt.legend(loc="best")
plt.subplot(212)
plt.title("Loss")
plt.plot(history.history["loss"], color="g", label="Train")
plt.plot(history.history["val_loss"], color="b", label="Validation")
plt.legend(loc="best")
plt.tight_layout()
plt.show()